In [61]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()
test.head()

#train.isnull().sum().head(41)
#train.isnull().sum().tail(40)
#train.columns

#test.isnull().sum().head(41)
#test.isnull().sum().tail(40)
#test.columns

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [62]:
# for the following 14 features we can replace NA with no as defined in description

missFeatures_na = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2','FireplaceQu',
                'GarageType', 'GarageFinish', 'GarageQual','GarageCond', 'PoolQC', 'Fence','MiscFeature']

missFeatures = train.columns[train.isnull().any()]

missFeatures = ['LotFrontage','Alley', 'MasVnrType','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure', 'BsmtFinType1','BsmtFinType2','Electrical',
 'FireplaceQu', 'GarageType','GarageYrBlt', 'GarageFinish','GarageQual','GarageCond','PoolQC','Fence', 'MiscFeature']


def ReplaceNaNo(cols):
    if pd.isnull(cols):
        return 'No'
    else:
        return cols


def featureEngineering(featuresWithNa, data):
    
    missFeatures  = data.columns[data.isnull().any()]
    features_na   = [x for x in missFeatures if x in featuresWithNa]
    features_null = [x for x in missFeatures if x not in featuresWithNa]
    
    catFeatures = data.select_dtypes(include ='object')
    label_encoder = LabelEncoder()
    
    for feature in features_na:
        data[feature] = data[feature].apply(ReplaceNaNo)
     
    for feature in features_null:
        data[feature].fillna(data[feature].mode()[0], inplace = True)
    
    for feature in catFeatures:
        data[feature] = label_encoder.fit_transform(data[feature].values)
    
    return data


trainSet = featureEngineering(missFeatures_na,train)
trainSet.isnull().sum()
#trainSet.iloc[:,:20].head()
#trainSet.iloc[:,20:40].head()
#trainSet.iloc[:,40:60].head()
#trainSet.iloc[:,60:].head()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [63]:
trainSet.shape
y = trainSet['SalePrice']
x = trainSet.drop(['SalePrice'],axis = 1)
x.shape,y.shape

(1460, 81)

((1460, 80), (1460,))

In [64]:
from sklearn.feature_selection import RFE
import time
model1 = GradientBoostingRegressor()
#model2 = RandomForestRegressor()

start = time.time()
rfes = []
for n in [15,20,25,30,35,40]:
    rfe = RFE(model1,n)
    rfes.append(rfe.fit(x,y))

end = time.time()
minutes = 1000*60
start/minutes, end/minutes

(26561.797232711284, 26561.800215000567)

In [65]:
from sklearn.model_selection import cross_val_score
import numpy as np
kf = KFold(n_splits=10,random_state=None)
features = []
for rfe in rfes:
    f = rfe.get_support(1)
    X = trainSet[trainSet.columns[f]]
    features.append(f)
    score = cross_val_score(model1,X,y,cv = kf ,scoring="neg_root_mean_squared_error")
    f.size,abs(score.mean())

(15, 26432.769859459186)

(20, 25721.391273653004)

(25, 24872.123654881194)

(30, 25348.562111188174)

(35, 25041.383478359516)

(40, 25043.239734316907)

In [54]:
selectedTrainFeatures = trainSet.iloc[:,features[1]]
#selectedTrainFeatures.head()

selectedTestSet = test[selectedTrainFeatures.columns]
#selectedTestSet.head()

selectedTestSet = featureEngineering(missFeatures_na,selectedTestSet)
selectedTestSet.head()

#selectedTestSet['GarageYrBlt'].value_counts()

/Users/ranjithtalamanchi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/ranjithtalamanchi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,MSZoning,LotArea,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtQual,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,KitchenQual,Fireplaces,GarageType,GarageCars,GarageArea,SaleCondition
0,2,11622,12,5,6,1961,1961,4,468.0,882.0,896,0,896,1,3,0,1,1.0,730.0,4
1,3,14267,12,6,6,1958,1958,4,923.0,1329.0,1329,0,1329,1,2,0,1,1.0,312.0,4
2,3,13830,8,5,5,1997,1998,2,791.0,928.0,928,701,1629,2,3,1,1,2.0,482.0,4
3,3,9978,8,6,6,1998,1998,4,602.0,926.0,926,678,1604,2,2,1,1,2.0,470.0,4
4,3,5005,22,8,5,1992,1992,2,263.0,1280.0,1280,0,1280,2,2,0,1,2.0,506.0,4


In [66]:
from sklearn import metrics
model1.fit(selectedTrainSet,y)
pred = model1.predict(selectedTestSet)
sv = pd.DataFrame({'Id' : test['Id'],'SalePrice': pred})

sub = pd.read_csv('sample_submission.csv')
sub.head()
sv.head()

abs(sv['SalePrice']-sub['SalePrice']).mean()

np.sqrt(metrics.mean_squared_error(sv['SalePrice'],sub['SalePrice']))

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


,Id,SalePrice
0,1461,128008.389291
1,1462,157076.748167
2,1463,175960.958347
3,1464,179950.724019
4,1465,193323.119539


54614.76331914824

72870.0564734496